In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
data = pd.read_csv('heart_disease_train_nomissing (1).csv')
data = data.drop('HeartDiseaseorAttack',axis=1)
data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,30.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0
1,1.0,1.0,1.0,22.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,30.0,0.0,1.0,0.0,12.0,4.0,4.0
2,1.0,1.0,1.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,0.0,0.0,1.0,1.0,10.0,6.0,5.0
3,1.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,4.0,10.0,17.0,1.0,0.0,9.0,4.0,1.0
4,1.0,1.0,1.0,33.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,1.0,0.0,0.0,1.0,11.0,6.0,6.0


In [6]:
from sklearn.utils import resample #sample 10,000 from dataset
sample_data = resample(data,
             replace=True,
             n_samples=10000,
             random_state=42)

print(sample_data.shape)

(10000, 21)


In [8]:

test = resample(data, # take exact same sample and store in test (this will be what we compare imputation to)
             replace=True,
             n_samples=10000,
             random_state=42)

test.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
121958,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,5.0,6.0
146867,0.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,4.0,5.0,30.0,0.0,1.0,10.0,4.0,1.0
131932,0.0,1.0,1.0,24.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0
103694,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,4.0,20.0,2.0,1.0,0.0,10.0,4.0,3.0
119879,1.0,0.0,1.0,34.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,10.0,0.0,0.0,1.0,5.0,6.0,8.0


In [9]:
#randomly replace 10% of rows with Nan
for col in sample_data.columns:
    sample_data.loc[sample_data.sample(frac=0.05).index, col] = np.nan

In [10]:
sample_data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
121958,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,5.0,6.0
146867,0.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,4.0,5.0,30.0,0.0,1.0,10.0,4.0,1.0
131932,0.0,1.0,1.0,24.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0
103694,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,4.0,20.0,2.0,1.0,0.0,10.0,4.0,3.0
119879,NaN,0.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,1.0,...,NaN,0.0,2.0,10.0,0.0,NaN,1.0,5.0,6.0,8.0


In [11]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()
sample_data = pd.DataFrame(scaler.fit_transform(sample_data), columns = sample_data.columns)

In [13]:
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

In [17]:
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(sample_data),columns = sample_data.columns)

In [18]:
np.nan in df.values #no more nan values

False

In [30]:
df.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,0.250000,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.833333,0.8,0.714286
1,0.0,1.0,1.0,0.315217,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.75,0.166667,1.000000,0.0,1.0,0.750000,0.6,0.000000
2,0.0,1.0,1.0,0.260870,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.00,0.000000,0.000000,0.0,1.0,0.583333,1.0,0.857143
3,1.0,1.0,1.0,0.293478,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.75,0.666667,0.066667,1.0,0.0,0.750000,0.6,0.285714
4,0.0,0.0,1.0,0.300000,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.25,0.333333,0.000000,0.0,1.0,0.333333,1.0,1.000000


https://stefvanbuuren.name/fimd/sec-evaluation.html - How to evaluate imputation methods

Now df has our imputed scaled values, test has the original scaled. Time to compare

In [31]:
comparison_df = pd.DataFrame()
comparison_df


""


In [32]:
for col in df.columns:
    comparison_df[col]=np.where(df[col] == test[col], 0, (df[col] - test[col])**2) # store squared difference

In [33]:
comparison_df.shape

(10000, 21)

In [34]:
comparison_df.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.069565,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
totals = list()
for col in df.columns:
    totals.append(comparison_df[col].sum())

In [40]:
totals

[204.0,
 216.80000000000004,
 34.6,
 30.52173913043478,
 233.20000000000002,
 47.4,
 18.6,
 149.39999999999998,
 210.8,
 145.2,
 56.2,
 39.0,
 62.0,
 100.9,
 68.0,
 74.10666666666665,
 95.4,
 239.4,
 101.35,
 76.04000000000002,
 105.11428571428573]

Using root mean square error

In [43]:
rmse = (sum(totals)/10000)**0.5

In [44]:
rmse

0.48041988838009064